In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
from src.gene_prediction.gene_prediction import read_data
import sklearn
import pandas as pd
import numpy as np

In [6]:
x_train, y_train, x_test = read_data()

Loading data...
All files loaded (15 s). Preprocessing...
Train / test data has 15485 / 3871 genes.
x_train shape is (15485, 500)
y_train shape is (15485,)
x_test shape is (3871, 500)
Data preprocessing done...
Next steps FOR YOU:
------------------------------
1. Define a classifier using sklearn
2. Assess its accuracy using cross-validation (optional)
3. Fine tune the parameters and return to 2 until happy (optional)
4. Create submission file. Should be similar to y_train.csv.
5. Submit at kaggle.com and sit back.


In [53]:
x_train_split, x_val_split, y_train_split, y_val_split = sklearn.model_selection.train_test_split(
    x_train,
    y_train,
    stratify=y_train,
    random_state=1238,
    test_size = 0.33
)

In [61]:
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline


In [73]:
lr1 = LogisticRegression(max_iter=10000, random_state=42, class_weight='balanced')
gb = GradientBoostingClassifier(random_state=42, max_depth=4, n_estimators=100)
la = Lasso(normalize=True, max_iter=5000, random_state=42)
mlp = MLPClassifier(random_state=42, early_stopping=True, max_iter=500)
lsvc = LinearSVC(class_weight='balanced', random_state=42, max_iter=5000)
svc = SVC(random_state=42, probability=True, class_weight='balanced')
voting_clf = VotingClassifier(
    [
        ('LR', lr1),
        ('GB', gb),
        ('MLP', mlp),
    ],
    voting='soft')
voting_clf.fit(x_train_split, y_train_split)

/Users/maycownmiranda/anaconda3/envs/pattern_rec_ime_usp/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


VotingClassifier(estimators=[('LR',
                              LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=10000,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=42, solver='warn',
                                                 tol=0.0001, verbose=0,
                                                 warm_start=False)),
                             ('GB',
                              GradientBoostingClassifier(criterion='friedman_mse',
                                                         init=None,
                                                         learn...
                                

In [74]:
y_hat_class_train_split = voting_clf.predict(x_train_split)
y_hat_class_val_split = voting_clf.predict(x_val_split)

y_hat_proba_train_split = voting_clf.predict_proba(x_train_split)
y_hat_proba_val_split = voting_clf.predict_proba(x_val_split)

print("AUC (Train) : {}".format(sklearn.metrics.roc_auc_score(y_train_split, y_hat_proba_train_split[:,1])))
print("AUC (Val): {}".format(sklearn.metrics.roc_auc_score(y_val_split, y_hat_proba_val_split[:,1])))

print(sklearn.metrics.classification_report(y_train_split, y_hat_class_train_split))
print(sklearn.metrics.classification_report(y_val_split, y_hat_class_val_split))


AUC (Train) : 0.95654457563053
AUC (Val): 0.9114623401362211
              precision    recall  f1-score   support

         0.0       0.88      0.90      0.89      5177
         1.0       0.90      0.88      0.89      5197

    accuracy                           0.89     10374
   macro avg       0.89      0.89      0.89     10374
weighted avg       0.89      0.89      0.89     10374

              precision    recall  f1-score   support

         0.0       0.85      0.84      0.85      2551
         1.0       0.85      0.85      0.85      2560

    accuracy                           0.85      5111
   macro avg       0.85      0.85      0.85      5111
weighted avg       0.85      0.85      0.85      5111

